In [1]:
!pip install neptune-client

     |████████████████████████████████| 92kB 3.8MB/s 
     |████████████████████████████████| 829kB 9.5MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 204kB 29.9MB/s 
     |████████████████████████████████| 163kB 31.7MB/s 
     |████████████████████████████████| 143kB 27.7MB/s 
     |████████████████████████████████| 133kB 42.1MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
     |████████████████████████████████| 71kB 6.3MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.4.119-py2.py3-none-any.whl size=150019 sha256=a6699533c81adb616d4ccd63c7477ac87d6a60bc851e00d0bb774b5e2d65e198
  Stored in directory: /root/.cache/pip/wheels/84/5c/c1/a81e80761b94b4467fd3fda1fd3109463702f6247fc422eb33
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=1cb12b8d8f651b3d378bd7efa4f6c7da95c4f936b2afba6f03c2cdb412e1416c
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a93

In [61]:
import neptune
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
sns.set(style="darkgrid")

project = neptune.init(
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNGIzMTllNDQtZGVjMi00MjcxLThiOTAtYTcxNzJiNWI5NjkzIn0=",
            project_qualified_name="elangenhan/hcd-experiments"
        )

In [121]:
experiments = project.get_experiments()

logs_names_deprecated = ['dur', 'train_loss', 'train_acc', 'train_f1', 'train_roc_auc', 'train_precision', 'train_recall', 'test_acc' , 'test_f1' , 'test_roc_auc' , 'test_precision' , 'test_recall']
logs_names            = ['dur', 'train_loss', 'train_acc', 'train_f1', 'train_roc_auc', 'train_precision', 'train_recall', 'valid_acc', 'valid_f1', 'valid_roc_auc', 'valid_precision', 'valid_recall', 'valid_loss']


counter = 1
length = len(experiments)
metrics_df = pd.DataFrame(columns=['id', *logs_names])
for experiment in experiments[::-1]:
  print(f'{counter}/{length} - {experiment.id} \r')
  counter = counter + 1
  if experiment.state == 'succeeded' and experiment.get_parameters()['max_epochs'] and experiment.get_logs()['dur']['lastX'] == 30.0:
    if int(experiment.id.split('-')[1]) < 114:
      df = experiment.get_numeric_channels_values(*logs_names_deprecated)
      df = df.rename(columns={'test_acc': 'valid_acc', 'test_f1': 'valid_f1', 'test_roc_auc': 'valid_roc_auc', 'test_precision': 'valid_precision', 'test_recall': 'valid_recall'})
      df.insert(loc=12, column='valid_loss', value="NA")
    else:
      df = experiment.get_numeric_channels_values(*logs_names)

    # net_type vgg 11 etc net + number
    net_type = experiment.get_parameters()["module"].split('\'')[1].split('.')[1].split('Pretrained')[0]
    df.insert(loc=13, column='net_type', value=net_type)

    # net_family vgg only net
    match = re.match(r"([a-z]+)([0-9]+)", experiments[0].name, re.I)
    if match: items = match.groups()
    df.insert(loc=13, column='net_family', value=items[0])

    # pretrained - boolean 0 oder 1 if pretrained
    pretrained = "not pretrained"
    if "pretrained" in experiment.name.lower(): pretrained = "pretrained"
    df.insert(loc=14, column='pretrained', value=pretrained)

    # normalized - boolean 0 oder 1 if normalized
    normalized = "not normalized"
    if "normalized" in experiment.name.lower(): normalized = "normalized"
    df.insert(loc=15, column='normalized', value=normalized)

    # full_data - boolean 0 oder 1 if full data set used
    full_data = ""
    if hasattr(experiment.get_logs(), 'test_name'):
      if "small" in experiment.get_logs()['test_name'].y:
        full_data = "partial data"
      else:
        full_data = "full data"
    else:
      if "small" in experiment.name.lower():
        full_data = "partial data"
      else:
        full_data = "full data"

    df.insert(loc=16, column='full_data', value=full_data)

    df.insert(loc=0, column='id', value=experiment.id)
    metrics_df = metrics_df.append(df, sort=True)

metrics_df.to_csv (r'./neptune_logs.csv', index = False, header=True)
print("done saving")

1/109 - HCDEX-304 
2/109 - HCDEX-303 
3/109 - HCDEX-302 
4/109 - HCDEX-301 
5/109 - HCDEX-300 
6/109 - HCDEX-299 
7/109 - HCDEX-298 
8/109 - HCDEX-297 
9/109 - HCDEX-296 
10/109 - HCDEX-294 
11/109 - HCDEX-293 
12/109 - HCDEX-292 
13/109 - HCDEX-291 
14/109 - HCDEX-290 
15/109 - HCDEX-289 
16/109 - HCDEX-288 
17/109 - HCDEX-287 
18/109 - HCDEX-286 
19/109 - HCDEX-285 
20/109 - HCDEX-284 
21/109 - HCDEX-283 
22/109 - HCDEX-282 
23/109 - HCDEX-281 
24/109 - HCDEX-280 
25/109 - HCDEX-279 
26/109 - HCDEX-278 
27/109 - HCDEX-277 
28/109 - HCDEX-276 
29/109 - HCDEX-275 
30/109 - HCDEX-174 
31/109 - HCDEX-172 
32/109 - HCDEX-167 
33/109 - HCDEX-158 
34/109 - HCDEX-157 
35/109 - HCDEX-156 
36/109 - HCDEX-152 
37/109 - HCDEX-147 
38/109 - HCDEX-146 
39/109 - HCDEX-145 
40/109 - HCDEX-144 
41/109 - HCDEX-143 
42/109 - HCDEX-142 
43/109 - HCDEX-141 
44/109 - HCDEX-140 
45/109 - HCDEX-139 
46/109 - HCDEX-138 
47/109 - HCDEX-137 
48/109 - HCDEX-136 
49/109 - HCDEX-135 
50/109 - HCDEX-134 
51/109 - 

In [117]:
experiments[45].get_parameters()['max_epochs']

30.0